In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.optimize as op
from scipy import stats
import emcee
import corner
from scipy.stats import chisquare
%matplotlib inline

/usr/local/lib/python2.7/dist-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [2]:
df=pd.read_csv("NED26.05.1-D-12.1.0-20160501.csv",skiprows=12)
df=df[np.isnan(df['redshift (z)'])]
df=df[df['D (Mpc)']<500]
df=df[np.isfinite(df.err)]
df=df[df['D (Mpc)']>35]

In [3]:
for i in np.unique(df.Method):
    df1=df[df.Method==i]
    print i,df1.shape

AGN time lag (24, 15)
Black Hole (12, 15)
CMD (3, 15)
Cepheids (4, 15)
D-Sigma (815, 15)
Dwarf Galaxy Diameter (6, 15)
FP (11038, 15)
Faber-Jackson (527, 15)
GCLF (36, 15)
GeV TeV ratio (3, 15)
H I + optical distribution (1, 15)
HII region diameter (1, 15)
IRAS (2089, 15)
Maser (6, 15)
Ring Diameter (71, 15)
SBF (236, 15)
SGRB (5, 15)
SNII optical (319, 15)
SNII radio (6, 15)
SNIa (1988, 15)
SZ effect (5, 15)
Sosies (129, 15)
Tertiary (92, 15)
Tully est (264, 15)
Tully-Fisher (36718, 15)


In [4]:
listcut=np.unique(df1['Galaxy ID'])
listcut

array(['2MASX J00024069+0846251', '2MASX J00041398-5113247',
       '2MASX J00062236-5010027', ..., 'WISEP J041245.99+294612.5',
       'WKK 2029', 'ZwCl 1115.2+3013 03'], dtype=object)

In [9]:
%%time
errlist=np.zeros((len(listcut),8))
k=0
ncutoff=10
nbins=1
for i in listcut[:100]:
    dfilter=(df1['Galaxy ID']==i)
    if(len(df1[dfilter])>ncutoff): # Select galaxies which have more than n measurements
        data=np.array([])
        dummy=df1[dfilter]
        for i in dummy.D:
            otherfilter=(dummy.D==i)
            # generate distance distribution for each distance modulus/ distance modulus error measurement
            AA=10**(np.random.normal(dummy['m-M'][otherfilter],dummy['err'][otherfilter],nbins)/5.+1)
            data=np.hstack((data,AA))            
        dumerr=disterr[dfilter]
        em=len(df1[dfilter])
        r = gmm.fit(data[:, np.newaxis])
        # Distances in Mpc
        errlist[k,0]=r.means_[0, 0]/1e6 # Centroid of the GMM-fitted distribution
        errlist[k,1]=em # Number of measurements for each galaxy
        errlist[k,2]=np.sqrt(r.covars_[0, 0])/1e6 # Variance of the GMM-fitted distribution
        errlist[k,3]=np.percentile(data, 50)/1e6 # Percentiles of the combined distance distributions (50, 84, 16):
        errlist[k,4]=np.percentile(data, 84)/1e6-errlist[k,3]
        errlist[k,5]=errlist[k,3]-np.percentile(data, 16)/1e6
        errlist[k,6]=dumerr.mean() # average reported error (Delta)
        errlist[k,7]=dummy['D (Mpc)'].mean() # average reported distance (D)
    k+=1


CPU times: user 220 ms, sys: 0 ns, total: 220 ms
Wall time: 219 ms


In [8]:
errlist.shape

(9123, 8)